In [ ]:
#===========================================================================================
# ElasticSearch 텍스트 임베딩 테스트 예제
# - 문장들을  추출 요약해서 요약문장을 만들고, 요약 문장의 평균을 구하여 문장 embedding을 생성하여 ES에 인덱스에 vector 추가하고, 검색하는 예제임
# - 말뭉치는 ai_hub에 원천말뭉치인 ts1 말뭉치에 tilte, content를 추출하여, content 요약문과 title에 대해 각각 vector를 만들어서 ES 인덱스로 추가하는 예시임.
#
# => 대규모 웹데이터 기반 한국어 말뭉치 데이터 
# 말뭉치 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=624

# -여기서는 elasticsearch 7.17.3 때를 기준으로 설명함.
# -** 따라서 elasticsearch python 모듈도 7.17.3 을 설치해야 함
# - elasticsearch 모듈 8.x 부터는 구문의 많이 변경되었음.
# - 예 : index 생성:  body로 모든 변수들를 지정하는 데시, 명시적으로 모든 변수들을 최상으로 지정해 줘야함.
# => 참고: https://towardsdatascience.com/important-syntax-updates-of-elasticsearch-8-in-python-4423c5938b17   

# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# => 데이터로 고차원벡터를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index_1.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#
#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transformers

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch==7.17

# 한국어 문장 분리기(kss) 설치
#!pip install kss

# 추출 요약 설치
#!pip install bert-extractive-summarizer

In [1]:
import torch
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder

import kss
import numpy as np
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers


# FutureWarning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, getListOfFiles
device = GPU_info()
#device = torch.device('cpu')

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 토큰 임베딩 관련 param
IS_EMBED_DIVIDE = True      #여기서는 토큰 임베딩은 True고정, True=문단의 여러 문장을, 토큰 단위로 분리후 벡터 구해서 인덱스 만듬/False=문단의 여러문장을 하나의 벡터를 구해서 인덱스 만듬.
EMBED_DIVIDE_LEN = [5,7,9]  #5 # 문장을 몇개(토큰)으로 분리할지 (7,8,10) 일때 성능 좋음=>50.8%, (5,7,9) 일때 차원축소 128=>41.80%(81.8%) 성능 좋음
MAX_TOKEN_LEN = 40          # 최대 몇개 token까지만 임베딩 할지

# 차원 축소 관련 param
# 차원 축소 할지, 768 값을 앞에 384 만큼만 배열 resize 시킴.  
# - 384로 줄일대 -2% 성능 저하 발생(512: -1.2%, 256: -6%)
DIM_RESIZE_METHOD = 2  # 0=차원축소 안함/1=resize 로 차원축소/2=Dense 모델 이용 차원축소
DIM_RESIZE_LEN = 128

# ONNX 모델 사용
IS_ONNX_MODEL = False        # True=onnx 모델 사용
#------------------------------------------------------------------------------------

seed_everything(seed)

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.130:9200/")
#es.info()


F:\AnacondaEnv\daEnv\bong\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
F:\AnacondaEnv\daEnv\bong\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
F:\AnacondaEnv\daEnv\bong\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


False
device: cpu


In [2]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
from myutils import bi_encoder, dense_model

# bi_encoder 모델 로딩
bi_encoder_path = "bongsoo/klue-sbert-v1"
pooling_mode = 'mean' # bert면=mean, albert면 = cls
 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128 if DIM_RESIZE_METHOD == 2 else 0 if DIM_RESIZE_METHOD != 2 else None
    
word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
print(f'---bi_encoder---------------------------')   
print(bi_encoder1)
print(word_embedding_model1)

print(f'---dense param---------------------------')   
# 출력 값 차원 축소 지정인 경우, token_embeddings 일때는 sentencebert 라이브러리를 이용하여 dense_model 모델 추가할수 없으므로,
# 사용자정의 dense_model을 정의해서, 가중치와 bias를 bi_encoder모델에 것을 얻어와서 적용해서 차원 죽소함.
# => resize 방식 보다 성능 떨어지지만, 128일때는 더 성능이 좋음
if DIM_RESIZE_METHOD == 2:
    #-------------------------------------------------------------------------
    # 처음에는 아래 코드를 활용하여 해당 모델의 128 weight와 bias를 저장해 두어야 함.
    #state_dict = bi_encoder1.state_dict()  # bi_encoder모델이 state_dict 얻어옴
    #print(state_dict.keys())
    #dense_weight = state_dict['2.linear.weight'] # denser 모델에 bi_encoder모델이 linear.weight 적용
    #dense_bias = state_dict['2.linear.bias']     # denser 모델에 bi_encoder모델이 linear.bias 적용
    
    # 처음에  weigth, bias 파일을 저장해 둠.
    #torch.save(dense_weight, 'klue-sbert-v1-weigth.pt')
    #torch.save(dense_bias, 'klue-sbert-v1-bias.pt')
    #-------------------------------------------------------------------------
    # weigth, bias 저장해둔 파일 로딩
    dense_weight = torch.load('../embedding_sample/faiss/data/dense_weight/klue-sbert-v1-weight-128.pt')
    dense_bias = torch.load('../embedding_sample/faiss/data/dense_weight/klue-sbert-v1-bias-128.pt')

    print('*dense_weight:{}'.format(dense_weight.size()))
    print(f'*dense_bias:{dense_bias.size()}')
 
# onnx 모델 로딩
if IS_ONNX_MODEL == True:
    onnx_model_path = "bongsoo/klue-sbert-v1-onnx"#"bongsoo/klue-sbert-v1-onnx"
    onnx_tokenizer, onnx_model = onnx_model(onnx_model_path)
    print(f'---onnx_model---------------------------')
    print(onnx_model)

---bi_encoder---------------------------
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 128, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)
Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
---dense param---------------------------
*dense_weight:torch.Size([128, 768])
*dense_bias:torch.Size([128])


In [3]:
from myutils import embed_text, onnx_embed_text

# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragrphs:list):
    if IS_ONNX_MODEL == True:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs) 
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs, token_embeddings=False)
    else:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, token_embeddings=True, return_tensor=False)
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, return_tensor=False)  
            
    return embeddings

In [ ]:
from tqdm.notebook import tqdm
import kss
from myutils import embed_text, divide_arr_avg_exten, clean_text

# 인덱싱 함수 
def index_data():
    es.indices.delete(index=INDEX_NAME, ignore=[404])
    count = 0
    # 인덱스 생성
    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
        count += 1
        #print(f'{count}:{source}') # 인덱스 구조 출력
        es.indices.create(index=INDEX_NAME, body=source)
        
    # json 파일들이 있는 폴더에 .json 파일 이름들을 얻기
    # =>DATA_FOLDER: .JSON파일들이 있는 폴더
    files = getListOfFiles(DATA_FOLDER)
    assert len(files) > 0 # files가 0이면 assert 발생
    print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))
 
    for idx, file in enumerate(tqdm(files)):
        if ".json" not in file:  #.json 파일이 아니면 합치지 않음
            continue
            
        count = 0
        docs = []
    
        # json 파일 로딩 => [SJML][text] entry만 불러옴
        json_data = json.load(open(file, "r", encoding="utf-8"))['SJML']['text']
        for data in json_data:
        #for data in json_data:
            count += 1
            doc = {} #dict 선언
            
            doc['title'] = data['title']            # 제목 설정
            doc['paragraph'] = data['content']      # 문장 담음.
                
            docs.append(doc)
            #print(f'count:{count}')
            #print(doc['title'])
            
            if count % BATCH_SIZE == 0:
                index_batch(docs)
                docs = []
                print("Indexed {} documents.".format(count))
                  
            # ** 10 개만 보냄
            #if count >= 10:
            #   break
            
        if docs:
            index_batch(docs)
            print("Indexed {} documents.".format(count))   
            
        es.indices.refresh(index=INDEX_NAME)
            
    es.indices.refresh(index=INDEX_NAME)
    #print("=== End Done indexing===")
                   

def index_batch(docs):
    
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        title = doc['title']
        paragraph = doc['paragraph']
        #sub_contexts = []
        #------------------------------------------------------------------------------------------------------------------------
        paragraph = clean_text(paragraph)  # 전처리 : (한글, 숫자, 영문, (), {}, [], %, ,,.,",')  등을 제외한 특수문자 제거
        # 입력 문단을 여러 문장들로 나눔.
        #sentences = [sentence for sentence in paragraph.split('.') if sentence != '' and len(sentence) > 10]  # '.'(마침표) 로 구분해서 sub 문장을 만듬.
        sentences = [sentence for sentence in kss.split_sentences(paragraph) if sentence != '' and len(sentence) > 10] # kss 이용해서 sub 문장을 만듬
        
        # 만약 sentences(sub 문장) 가 하나도 없으면 원본문장을 담고, 10이상이면  10개만 담음.
        sub_contexts.append([context] if len(sentences) < 1 else sentences[0:10] if len(sentences) > 10 else sentences)
       
        if i < 1:
            print(sub_contexts)
        
        #------------------------------------------------------------------------------------------------------------------------
        # 토큰 분할 임베딩 처리
        token_embeds = embedding(sub_contexts)
        #------------------------------------------------------------------------------------------------------------------------ 
        # 토큰 분할인 경우 처리 start=>           
        token_embed_arr_list = []
        tcount = 0
        # tensor(250,768) 한문장 토큰 임베딩 얻어와서, 각 ?개 토큰씩 평균을 구함.
        for token_embed in token_embeds:
            token_embed = token_embed[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
            if tcount >= MAX_TOKEN_LEN: 
                break
                
            token_embed_arrs = token_embed.cpu().numpy().astype('float32')
            #print(f'token_embed_arrs:{token_embed_arrs.shape}')
            # 5,7,10 씩 자르면서 문장 토큰 평균을 구함
            token_embed_divide_arrs = divide_arr_avg_exten(embed_arr=token_embed_arrs, divide_arrs=EMBED_DIVIDE_LEN) 

             # Dense 방식으로 차원 축소 => 평균 구한후 차원 축소하는 방식이 0.6% 정도 성능 좋음
            if DIM_RESIZE_METHOD == 2:
                tmp1 = torch.Tensor(token_embed_divide_arrs)
                #tmp1 = torch.from_numpy(token_embed_divide_arrs)
                debug1 = False
                tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias, debug=debug1)
                arrs = tmp2.detach().numpy().astype('float32')
            else:  
                arrs = token_embed_divide_arrs
                
            # 평균 구한 토큰들을 token_embed_arr_list 리스트에 담아둠.(50보다 크면 50개만 담음)           
            for idx, arr in enumerate(arrs):
                
                 # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
                if DIM_RESIZE_METHOD == 1:
                    darr = np.resize(arr, (DIM_RESIZE_LEN,))
                else:
                    darr = arr
                    
                token_embed_arr_list.append(darr)
                tcount +=1
                if tcount >=MAX_TOKEN_LEN:
                    break

        #embeddings = np.array(token_embed_arr_list)
        #------------------------------------------------------------------------------------------------------------------------
        # ES에 문단 인덱싱 처리
        request = {}  #dict 정의
        request["rfile_name"] = title       # 제목               
        request["rfile_text"] = paragraph   # 문장
        
        request["_op_type"] = "index"        
        request["_index"] = INDEX_NAME
        
        # for문 돌면서 벡터 처리
        #print(type(token_embed_arr_list))
        #print(len(token_embed_arr_list))
        
        # vector 1~40 까지 값을 0으로 초기화 해줌.
        for i in range(MAX_TOKEN_LEN):
            if DIM_RESIZE_METHOD > 0:
                request["vector"+str(i+1)] = np.zeros((DIM_RESIZE_LEN))
            else:
                request["vector"+str(i+1)] = np.zeros((768))
            
        # vector 값들을 담음.
        for i, token_embed_arr in enumerate(token_embed_arr_list):
            request["vector"+str(i+1)] = token_embed_arr
            
        requests.append(request)
        #------------------------------------------------------------------------------------------------------------------------
                
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)
        

In [ ]:
#======================================================================================
# ElasticSearch(이하:ES) 데이터 인텍싱
# - ElasticSearch(이하:ES)에 KorQuAD_v1.0_train_convert.json 파일에 vector값을 구하여 index 함
#
# => index 명 : korquad
# => index 구조 : index_1.json 파일 참조
# => BATCH_SIZE : 100 => 100개의 vector값을 구하여, 한꺼번에 ES에 인텍스 데이터를 추가함.
#======================================================================================
INDEX_NAME = 'aihub-ts1-test-klue-sbert-v1-mpower10u-stoken-128d'  # ES 인덱스 명 (*소문자로만 지정해야 함)
INDEX_FILE = './data/mpower10u_128d.json'                 # 인덱스 구조 파일
DATA_FOLDER = '../../../data11/ai_hub/ts1/sample1/'     # 인덱스할 파일들이 있는 폴더경로 
BATCH_SIZE = 100

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info())

# 2. index 처리
index_data()

In [ ]:
# kibana 콘솔창에 접속해서 계수 확인
# http://192.168.0.130:5601/app/dev_tools 에 접속해서 해야함

## 입력 ##
# GET korquad/_count

## 출력 ###
'''
{
  "count" : 1420,
  "_shards" : {
    "total" : 2,
    "successful" : 2,
    "skipped" : 0,
    "failed" : 0
}
'''    

In [13]:
# 검색 하기

import time
from elasticsearch import Elasticsearch
from myutils import embed_text, divide_arr_avg_exten

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query():
    
    query = input("검색어 입력: ")
    
    start_embedding_time = time.time()
    
    # 문장으로 비교할때=>쿼리 문장에 대한 벡터 생성해서 비교
    #query_vector = embed_text(model=bi_encoder,paragraphs=[query])[0]
    #------------------------------------------------------------------
    # 토큰 평균으로 비교할때=> 쿼리 문장에 대한 모든 토큰 벡터를 생성해서 비교
    # 토큰 분할 임베딩 처리
    token_query_embeds = embedding([query])
    
    token_query_embed_arr_list = []
    # 쿼리 문장들의 토큰들의 평균을 구함.
    for token_query_embed in token_query_embeds:
        
        token_query_embed = token_query_embed[1:-1] # 맨앞에 [CLS]와 맨두에 [SEP] 토큰은 뺌
        print(f'token_query_embed.shape:{token_query_embed.shape}')
        
        tmp = token_query_embed.cpu().numpy().astype('float32')
        tmp=tmp.mean(axis=0) #평균 구함
        
         # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
        if DIM_RESIZE_METHOD == 1:
            tmp = np.resize(tmp, (DIM_RESIZE_LEN,))
         # Dense 방식으로 차원 축소=> 평균 구하기 전에 차원 축소하는것이 성능 +0.6더 좋음
        elif DIM_RESIZE_METHOD == 2:
            tmp1 = torch.Tensor([tmp]) # 1차원 배열을 -> 2차둰 텐서(1,768)로 변환
            tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
            tmp = tmp2.detach().numpy().astype('float32').ravel(order='C') # 1차원 배열로 변경(128,)        
        
        token_query_embed_arr_list.append(tmp)
        
    query_vector = np.array(token_query_embed_arr_list)[0]  # 리스트를 배열로 변환  
    
    # print(query_vector)
    print(f'query_vector.shape:{query_vector.shape}')
    #------------------------------------------------------------------
    
    end_embedding_time = time.time() - start_embedding_time
    
    # 쿼리 구성
    '''
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['vector2']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴(doc['summarize_vector'])
                "params": {"query_vector": query_vector}
            }
        }
    }
    '''
    # 문단별 50개의 벡터와 쿼리벡터를 서로 비교하여 최대값 갖는 문단들중 가장 유사한  문단 출력
    # => script """ 안에 코드는 java 임.
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}
            },
                "script":{
                    "source": """
                      float max_score = 0;
                      for(int i = 1; i <= params.VectorNum; i++) 
                      {
                          float[] v = doc['vector'+i].vectorValue; 
                          float vm = doc['vector'+i].magnitude;  
                          
                          float dotProduct = 0;
                          
                          for(int j = 0; j < v.length; j++) 
                          {
                              dotProduct += v[j] * params.queryVector[j];
                          }
                          
                          float score = dotProduct / (vm * (float) params.queryVectorMag);
                          
                          if(score > max_score) 
                          {
                              max_score = score;
                          }
                      }
                      return max_score
                    """,
                "params": 
                {
                  "queryVector": query_vector,
                  "queryVectorMag": 0.25357,
                  "VectorNum": 30
                }
            }
        }
    }
    
    #print('query\n')
    #print(script_query)
    
    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["rfile_name","rfile_text"]}
        }
    )
    end_search_time = time.time() - start_search_time
    
    print("{} total hits.".format(response["hits"]["total"]["value"])) 

        
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    # print(response)
    texts = []
    titles = [] 
    bi_scores = []
    for hit in response["hits"]["hits"]: 
        '''
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[요약문]')
        print(hit["_source"]["summarize"]) 
        print()
                
        '''
        #print(hit)
        
        # 리스트에 저장해둠
        titles.append(hit["_source"]["rfile_name"])
        texts.append(hit["_source"]["rfile_text"])
        bi_scores.append(hit["_score"])
        
     # 내림 차순으로 정렬
    dec_bi_scores = reversed(np.argsort(bi_scores))
    print(dec_bi_scores)
    
    # 내림차순으로 출력
    for idx in dec_bi_scores:
        print("{:.2f}\t[제목]:{}\n{}\n".format(float(bi_scores[idx]), titles[idx], texts[idx]))
    
    # 처리 시간들 출력
    print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
    print("search time: {:.2f} ms".format(end_search_time * 1000)) 
    print('\n')
     #========================================================================================================    
        

In [ ]:
#====================================================================
# ES 인덱싱된 내용 검색 
# => cosineSimilarity 스크립트를 이용하여 ES로 query 함(*이때 SEARCH_SIZE를 몇개로 할지 지정할수 있음)
# => 쿼리 응답 결과 값에서 _id, _score, _source 등을 뽑아냄
#====================================================================
INDEX_NAME = 'aihub-ts1-test-klue-sbert-v1-mpower10u-stoken-128d'  # ES 인덱스 명 (*소문자로만 지정해야 함)
#INDEX_NAME = 'korquad-klue-sbert-v1.0-noavg' # 요약문 평균값 처리 안한경우

SEARCH_SIZE = 10

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

# 2. query 처리
run_query_loop()

<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.27', 'port': 9200}])>>


검색어 입력:  갤럭시 s6


token_query_embed.shape:torch.Size([3, 768])
query_vector.shape:(128,)


C:\Users\A\AppData\Local\Temp\ipykernel_13540\1923550877.py:115: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(


7386 total hits.
2.77	[제목]:lgu+, iot 기술 ddp에 총동원한다
lg유플러스가 서울디자인재단과 손잡고 서울의 명소 동대문디자인플라자(ddp)에 '세계가 벤치마킹 할' iot(internet of thing, 사물인터넷) 플랫폼을 구현한다.. . lg유플러스는 iot 선도 기술력과 all-ip 네트워크를 기반으로 세계 최대 3차원 비정형 건물인 ddp에 iot 미래 기술을 6월부터 구축에 착수한다고 7일 밝혔다.. . lg유플러스는 연간 1000만명 국내·외 방문객을 눈앞에 두고 있는 ddp에 iot 솔루션을 도입, ▲비콘(beacon)기반의 고객 서비스 ▲무선 센서 기반의 관제 시스템 ▲비콘 및 결제 시스템을 이용한 지역 상권 확대에 나선다.. . ◇기다리지 않고 마음껏 즐겨라. . lg유플러스는 축구장 크기의 3배에 달하는 ddp에서 고객이 전시시설을 쉽게 찾을 수 있도록 비콘 기반의 위치정보 서비스를 제공할 계획이다. 비콘은 저전력 블루투스를 이용해 사람이나 사물의 위치를 파악하는 근거리 무선 통신 기술이다. nfc와 달리 근접하지 않아도 정보수신이 가능하며, gps로는 불가능했던 실내에서도 정확한 위치확인이 가능한 iot 기술이다.. . ddp 방문객은 비콘 기반의 '스마트 3d 도보 내비게이션' 앱으로 실내 사진을 이용해 전시 공간을 비롯한 ddp 내 목적지를 간편하게 찾을 수 있다. 주차장에서는 비콘으로 내 주차 위치를 스마트폰으로 확인하고 찾을 수 있는 '내차 위치 안내' 서비스, 전시관 바로 앞에서 (이름)용을 확인하고 바로 티켓을 구매할 수 있는 '모바일 발권 시스템', 본인의 스마트폰으로 작품 설명을 들을 수 있는 '스마트 비디오 도슨트', ddp 살림터에서 이벤트 정보를 확인하는 '모바일 카탈로그' 등 다양한 편의 서비스도 이용할 수 있다.. . ◇무선 센서(sensor)로 ddp 실시간 관제. . lg유플러스는 초대형 건축물인 ddp를 위한 무선 센서 기반의 관제 시스템을 전격 도입하기로 했다.. . ddp 곳곳에

검색어 입력:  sk텔레콤


token_query_embed.shape:torch.Size([4, 768])
query_vector.shape:(128,)
7386 total hits.
3.22	[제목]:탄핵 소통에 모바일 데이터 2배 '폭증'
(이름) 전 대통령에 대한 헌법재판소의 탄핵 인용이 결정된 10일, 헌재의 선고를 전후해 모바일 데이터 사용량이 두 배 이상 증가한 것으로 확인됐다. 이동통신사들은 탄핵 관련 집회 대비 차원에서 (이름)지국을 최대 5대까지 배치하는 등 준비에 들어갔다. . . 10일 sk텔레콤에 따르면 헌재의 탄핵 선고가 진행됐던 이날 오전 11시 전후 모바일 데이터 사용량이 평소의 2배가량 증가한 것으로 확인됐다.. . 이용자들이 모바일 기기를 이용해 생방송으로 진행된 탄핵 심판 선고 뉴스를 지켜 봤고, 이것이 데이터 사용량을 늘린 가장 큰 (이름) 것으로 추정된다. 아울러 모바일 메신저로 탄핵 관련 이야기를 주고받은 이용자들의 패턴 증가도 모바일 데이터 사용량에 영향을 미친 것으로 보인다. . . 이와 함께 kt와 lg유플러스의 이용자들도 정확한 수치가 공개되진 않았지만 평소보다 많은 데이터 사용량을 보인 것으로 알려졌다. . . 이동통신 3사는 이 날 오후 대통령 탄핵과 관련해 대규모 집회가 예상되는 서울 광화문과 종로, 시청 주변에 (이름)지국을 배치하는 등의 준비로 이용객들의 불편을 (이름)한다는 계산이다. . . 이용자가 가장 많은 sk텔레콤은 5대의 (이름)지국을 배치할 예정이다. 집회 예상지역 트래픽 수용량을 평소보다 4.5배 가량 늘렸고 lte(롱텀에볼루션)와 3g 기지국 400개도 추가 설치했다. . . kt도 3대의 (이름)지국을 운영한다. 기존 대비 운영시설을 2.6배 증설하고 긴급복구용 발전기 6대를 추가로 구성했다. 비상근무 인원 147명이 현재 대기 중인 상황이다. . . lg유플러스는 5대의 (이름)지국을 배치했다. 집회가 진행될 것으로 보이는 지역에 기지국 60대, 와이파이 중계기 13대를 설치했다. 평소 3배 (이름) 트래픽 수용이

검색어 입력:  kisdi 원장


token_query_embed.shape:torch.Size([5, 768])
query_vector.shape:(128,)
7386 total hits.
3.12	[제목]:청학동 어린 선비들, 태블릿 들고 과거시험을…
kt가 24일 경남 하동군 청암면 묵계리 청학동 기가창조마을 도서관 앞 마당에서 청학동 거주 어린이들을 대상으로 '청학동 it 과거시험'을 치렀다.. . 이날 행사는 지난 7월 구축한 '청학동 기가 창조마을' 개소 100일을 맞아 마련됐다. 청학동 거주 어린이 총 60여명이 '지필묵' 대신 태블릿 pc와 시험 전용 앱(애플리케이션)을 활용해 과거시험에 참여했다. . . 옛 선비들이 입었던 도포와 유건을 착용하고 참가 어린이들은 앱으로 풀어보는 '한자시험'과 사자소학을 암송하는 '강경대회', 상식을 겨루는 '도전! 청학 골든벨'등의 다양한 시험을 봤다. . . 이날 시험에서는 청학동 서당의 훈장과 kt와 지자체 관계자의 공정한 심사를 거쳐 장원, 방안, 탐화 등 총 10명의 어린이가 수상의 영광을 안았다. .

2.88	[제목]:'리베이트 과다 지급' skt, 1주간 신규모집금지
sk텔레콤이 대리점과 판매점이 주는 판매장려금(리베이트)를 과도하게 지급해 불법 보조금으로 사용할 수 있게 했다며 1주일간 신규모집 금지 조치를 받았다. 신규모집금지 시기는 차기 회의에서 결정한다.. . (이름)신위원회는 26일 전체회의를 열고 지난 1월1일부터 31일까지 sk텔레콤 유통점 38개의 가입자 2960건을 조사한 결과 2050명에게 페이백 등의 방법으로 평균 22만8000원의 보조금을 편법 지급했다며 신규모집금지 7일에 과징금 235억원을 부과했다.. . 방통위에 따르면 sk텔레콤은 1월17일부터 19일까지 갤럭시 노트4와 아이폰6 등 최신 단말기에 대해 최대 50만원 정도의 리베이트를 지급해 차별적 지원금을 유도한 것으로 조사됐다. . . 방통위는 1월16일부터 시장 상황이 과열된다고 보고 이통3사 마케팅 담당자를 불러 자제를 권고했으며, 시장이 진정되지

In [ ]:
#==============================================================================================
# ES index에 데이터 추가하가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
#===============================================================================================

# ES에 이미 생성된 index
INDEX_NAME = 'korquad'
BATCH_SIZE = 30


# 1.추가할 데이터 준비
title = [
    '제주도', 
    '한라산',
    '서울특별시'
        ]

paragraph = [
    '대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.',
    '대한민국에서 가장 큰 섬인 제주도에 있으며 대한민국의 실효지배 영토 내의 최고봉이자 가장 높은 산(해발 1,947m). 대한민국의 국립공원 중 하나이다. 국립공원 전역이 유네스코 세계유산으로 지정되었다.',
    '대한민국의 수도인 서울은 현대적인 고층 빌딩, 첨단 기술의 지하철, 대중문화와 예것이 공존하는 대도시. 주목할 만한 명소로는 초현대적 디자인의 컨벤션 홀인 동대문디자인플라자, 한때 7,000여 칸의 방이 자리하던 경복궁, 조계사가 있다',
            ]

# {'paragraph': "", 'title': ""}

# 2. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

doc = {}
docs = []
count = 0

# 3. batch 사이즈 만큼식 ES에 추가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
for title, paragraph in zip(title, paragraph):
    doc = {}
    doc['paragraph'] = paragraph
    doc['title'] = title
    docs.append(doc)
    count += 1
    if count % BATCH_SIZE == 0:
        index_batch(docs)
        docs = []
        print("Indexed {} documents.".format(count))
   
# docs 이 있으면 전송
if docs:
    index_batch(docs)
    print("Indexed {} documents(end).".format(count))


In [ ]:
#==============================================================================================
# ES 데이터 조회하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

###########################################################
# 인덱스내 데이터 조회 => query 이용
###########################################################
def search(index, data=None):
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    res = es.search(index=index, body=body)
    return res
###########################################################

# 모든 데이터 조회
#sr = search(index=INDEX_NAME)
#pprint.pprint(sr)

# 단일 필드 조회
sr = search(index=INDEX_NAME, data = {'title': '제주도'})
print(sr)


In [ ]:
#==============================================================================================
# ES index에 데이터 삭제하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 삭제 => query 이용
############################################################
def delete(index, data):
    if data is None:  # data가 없으면 모두 삭제
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {"query": data}
    return es.delete_by_query(index, body=body)

############################################################
## 2: 인덱스 내의 데이터 삭제 => id 이용
############################################################
def delete_by_id(index, id):
    return es.delete(index, id=id)

############################################################
## 3: 인덱스 자체 삭제
############################################################
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)


# 1: query 이용 데이터 삭제
delete(index=INDEX_NAME, data={'title':'한라산'})

# 3: 인덱스 자체 삭제
#delete_index(index=INDEX_NAME)


In [ ]:
#==============================================================================================
# ES index에 데이터 업데이트하기
#==============================================================================================
INDEX_NAME = 'korquad'

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info)

############################################################
## 1: 인덱스 내의 데이터 업데이트=>_id 에 데이터 업데이트
############################################################
def update(index, id, doc, doc_type):
    
    body = {
        'doc': doc
    }
    
    res=es.update(index=index, id=id, body=body, doc_type=doc_type)
    return res
############################################################

#=====================================================================
# 검색해서, _id, _type을 구함
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print('\n')
print("===[검색 결과]===")
for hits in sr["hits"]["hits"]:
    id = hits["_id"]      # id
    type = hits["_type"]  # type
    
    print(f'id: {id}')
    print(f'type: {type}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    print('\n')
    
    # update 시킴
    print("===[업데이트]===")
    doc = {'paragraph': '제주도는 대한민국에 가장 남쪽에 있는 섬으로, 인구는 약 71만명이며, 화산섬으로 관광자원이 많은 천혜의 관광지 이다.'}
    print(doc)
    print('\n')
    
    ur=update(index=INDEX_NAME, id=id, doc=doc, doc_type=type)
    print("===[업데이트 결과]===")
    print(ur)
    print('\n')

#=====================================================================

# 인덱스 refresh 함
# elasticsearch의 자동 새로고침의 시간은 1초 정도 소요
# 따라서 코드에 아래 명령어를 입력하지 않았을 경우 검색을 하지 못할 가능성도 존재
es.indices.refresh(index=INDEX_NAME)

# 제주도로 검색해서 한번더 확인
sr = search(index=INDEX_NAME, data = {'title': '제주도'})

print("===[재검색 결과]===")
for hits in sr["hits"]["hits"]:
    
    print(f'id:{hits["_id"]}')
    print(f'type: {hits["_type"]}')
    print(f'title:{hits["_source"]["title"]}')
    print(f'paragraph:{hits["_source"]["paragraph"]}')
    
              